This file is designed to work with Google Colab; for the data table, download the Excel file(df-ga-channel-group-report-monthly), update it as needed, and then upload and convert it to a Google Spreadsheet.

In [1]:
!pip install langchain-core langchain-openai langchain-anthropic langchain-google-genai
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.51
    Uninstalling langchain-core-0.3.51:
      Successfully uninstalled langchain-core-0.3.51
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 782.7 kB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.10.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [2]:
!pip install openai
!pip install reportlab
!pip install --upgrade reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.9 MB/s eta 0:00:00


In [3]:
from google.colab import auth, drive
import gspread
from google.auth import default
import pandas as pd
import os
# Googleシートを認証する
drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Googleシートのファイルを開ける
filename = "gsc-seo-keyword-performance-report"
ss = gc.open(filename)
# Googleシートのシートを読み込む
def load_sheet_data(sheet_name):
    worksheet = ss.worksheet(sheet_name)
    data = worksheet.get_all_values()
    return pd.DataFrame(data[1:], columns=data[0])

# Load data from all sheets and concatenate into one DataFrame
all_sheets = ss.worksheets()  # Get all sheet objects
df_all = pd.DataFrame()  # Initialize an empty DataFrame

for sheet in all_sheets:
    if sheet.title not in ["df_kw_target_jp", "df_kw_target_en"]:
        df = load_sheet_data(sheet.title)  # Load data from sheet
        df['DateRange'] = sheet.title  # Add a column to track the sheet name
        df_all = pd.concat([df_all, df], ignore_index=True)  # Combine data

df_all["Clicks"] = df_all["Clicks"].str.replace(',', '').astype(int)
df_all["Impressions"] = df_all["Impressions"].str.replace(',', '').astype(int)
df_all["CTR"] = df_all["CTR"].str.replace('%', '').astype(float) / 100  # Divide by 100 to convert to a proper fraction
df_all["Position"] = df_all["Position"].astype(float)

Mounted at /content/drive


In [5]:
# Generate Click Table
df_click = df_all.pivot_table(index="Query", columns="DateRange", values="Clicks", aggfunc="sum")
df_click = df_click.sort_values(by=df_click.columns[-1], ascending=False)
df_click = df_click.fillna(0).astype(int)

# Identify columns
last_col = df_click.columns[-1]         # e.g., "202503"
second_last_col = df_click.columns[-2]    # e.g., "202502"
last_year_col = str(int(last_col) - 100)    # e.g., "202503" -> "202403"

df_click['vs Last Month'] = df_click[last_col] - df_click[second_last_col]
df_click['vs Last Year'] = df_click[last_col] - df_click[last_year_col]

In [6]:
df_click.head()

DateRange,202402,202403,202501,202502,202503,vs Last Month,vs Last Year
Query,,,,,,,
スプレッドシート タスク管理 テンプレート 無料,0,0,115,133,136,3,136
スプレッドシート タスク管理 テンプレ,2,1,55,67,51,-16,50
wbs テンプレート スプレッドシート,0,0,10,10,21,11,21
スプレッドシート タスク管理 テンプレート,1,1,24,31,18,-13,17
タスク管理 スプレッドシート テンプレ,2,0,11,8,17,9,17


In [7]:
import traceback
import os
import time

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from google.colab import userdata

# Retrieve the OpenAI API key from user data
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Define the translation prompt template
PROMPT = """
Translate the following Japanese keyword to English, focusing strictly on the keyword without adding any extra terms unless part of the original keyword.
Return only the English version of the keyword, ensuring it directly corresponds to the input.

- Example:
- Input: 'スプレッドシート タスク管理 テンプレート 無料	'
- Expected Output: 'Spreadsheet Task Management Template Free'

Please process the keyword provided below:
- {keyword}
"""

# モデルを初期化する関数
def init_models(temperature=0):
    model_1= ChatOpenAI(temperature=temperature, model_name="gpt-4o-mini")
    return model_1

# チェーンの初期化
def init_chain():
    # モデルの初期化
    model_1 = init_models()

    # 各プロンプトと出力パーサーの設定
    prompt_1 = ChatPromptTemplate.from_messages([
        ("user", PROMPT),
    ])

    output_parser = StrOutputParser()

    # チェーンの構成
    chain_1 = prompt_1 | model_1 | output_parser

    return chain_1

# メインの関数
def translate_agent(keyword):
    # チェーンの初期化
    chain_1 = init_chain()

    if chain_1:

        try:
            # Step 1: 翻訳
            output = chain_1.invoke({
                "keyword": keyword
            })
            result = output.strip()
            time.sleep(3)  # 3秒の待機を追加

            return result

        except Exception as e:
            print("An error occurred during the process.")
            print(traceback.format_exc())

In [8]:
df_query = df_click.head(55).copy()
df_query["Query(En)"] = df_query.index.map(translate_agent)
df_query.reset_index(inplace=True)
df_query = df_query[['Query', 'Query(En)']]

# Combine Query and Click table
df_click_selected = df_query.merge(df_click, on="Query", how="left")

# Generate Impression Table
df_imp = df_all.pivot_table(index="Query", columns="DateRange", values="Impressions", aggfunc="sum")
df_imp = df_imp.sort_values(by=df_imp.columns[-1], ascending=False)
df_imp = df_imp.fillna(0).astype(int)
last_col = df_imp.columns[-1]         # e.g., "202503"
second_last_col = df_imp.columns[-2]    # e.g., "202502"
last_year_col = str(int(last_col) - 100)  # e.g., "202503" -> "202403"

df_imp['vs Last Month'] = df_imp[last_col] - df_imp[second_last_col]
df_imp['vs Last Year'] = df_imp[last_col] - df_imp[last_year_col]

df_imp_selected = df_query.merge(df_imp, on="Query", how="left")

# Generate CTR Table
df_ctr = df_all.pivot_table(index="Query", columns="DateRange", values="CTR", aggfunc="sum")
df_ctr = df_ctr.fillna(0).astype(float)
last_col = df_ctr.columns[-1]         # e.g., "202503"
second_last_col = df_ctr.columns[-2]    # e.g., "202502"
last_year_col = str(int(last_col) - 100)  # e.g., "202503" -> "202403"
df_ctr['vs Last Month'] = df_ctr[last_col] - df_ctr[second_last_col]
df_ctr['vs Last Year'] = df_ctr[last_col] - df_ctr[last_year_col]
df_ctr = df_ctr * 100
df_ctr = df_ctr.applymap(lambda x: f"{x:.1f}%")
df_ctr_selected = df_query.merge(df_ctr, on="Query", how="left")

# Generate Rank Table
df_rank = df_all.pivot_table(index="Query", columns="DateRange", values="Position", aggfunc="sum")
df_rank = df_rank.sort_values(by=df_rank.columns[-1], ascending=False)
df_rank = df_rank.fillna(0).astype(float)
last_col = df_rank.columns[-1]         # e.g., "202503"
second_last_col = df_rank.columns[-2]    # e.g., "202502"
last_year_col = str(int(last_col) - 100)   # e.g., "202503" -> "202403"
df_rank['vs Last Month'] = df_rank[last_col] - df_rank[second_last_col]
df_rank['vs Last Year'] = df_rank[last_col] - df_rank[last_year_col]

df_rank_selected = df_query.merge(df_rank, on="Query", how="left")

<ipython-input-8-37171bf0b248>:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_ctr = df_ctr.applymap(lambda x: f"{x:.1f}%")


In [10]:
########################################
# 1. Helper Function to Merge and Process
########################################
import numpy as np
import pandas as pd

def merge_category_data(
    base_df: pd.DataFrame,
    df_click: pd.DataFrame,
    df_rank: pd.DataFrame,
    excluded_columns=None,
    fill_zero_as_rank=100.0,
    average_suffix='Average',
    total_suffix='Total'
):
    """
    Merge a category-specific DataFrame (base_df) with main click & rank DataFrames,
    then add average/total rows.
    """
    if excluded_columns is None:
        excluded_columns = ['vs Last Month', 'vs Last Year', 'MSV']

    # ========== 1. クリックのマージ ==========
    merged_click = base_df.merge(df_click, on="Query", how="left").fillna(0)
    float_cols_click = merged_click.select_dtypes(include=['float']).columns
    merged_click[float_cols_click] = merged_click[float_cols_click].astype(int)

    # ========== 2. ランクのマージ ==========
    merged_rank = base_df.merge(df_rank, on="Query", how="left").fillna(0)

    # 日付形式(例: "202402", "202403", "202501"...)の列だけを特定
    rank_date_cols = [
        col for col in merged_rank.columns
        if col.isdigit() and col not in excluded_columns
    ]
    rank_date_cols.sort()  # 例: ["202402", "202403", "202501", "202502", "202503"]

    # (A) 先にランク列(0→100, NaN→100)を埋める
    merged_rank[rank_date_cols] = merged_rank[rank_date_cols].replace({
        0.0: fill_zero_as_rank,
        float('nan'): fill_zero_as_rank
    })

    # (B) "vs Last Month" の計算
    if len(rank_date_cols) >= 2:
        this_month_col = rank_date_cols[-1]  # 例: "202503"
        last_month_col = rank_date_cols[-2]  # 例: "202502"
        merged_rank['vs Last Month'] = merged_rank[this_month_col] - merged_rank[last_month_col]
    else:
        merged_rank['vs Last Month'] = 0.0

    # (C) "vs Last Year" の計算
    if len(rank_date_cols) >= 1:
        this_month_col = rank_date_cols[-1]  # 最新月
        # 例: "202503" -> "202403"
        last_year_col = str(int(this_month_col) - 100)
        if last_year_col in rank_date_cols:
            merged_rank['vs Last Year'] = merged_rank[this_month_col] - merged_rank[last_year_col]
        else:
            merged_rank['vs Last Year'] = 0.0
    else:
        merged_rank['vs Last Year'] = 0.0

    # ========== 3. 平均行の追加 ==========
    # 平均したい数値カラムを明示的に定義する
    # 例えば: rank_date_cols + ['vs Last Month', 'vs Last Year'] のようにする
    average_target_cols = rank_date_cols + ['vs Last Month', 'vs Last Year']

    # ただし、DataFrame の列に存在しない場合もあるのでフィルタリング
    average_target_cols = [c for c in average_target_cols if c in merged_rank.columns]

    # 平均を計算 (numeric_only=True を指定)
    mean_values = merged_rank[average_target_cols].mean(numeric_only=True)

    # 平均行を DataFrame 化
    mean_row = pd.DataFrame([mean_values], columns=mean_values.index)
    mean_row['Query'] = average_suffix
    mean_row['Query(En)'] = ' - '
    mean_row['MSV'] = ' - '

    # DataFrame に結合
    merged_rank = pd.concat([merged_rank, mean_row], ignore_index=True)

    # 数値カラムを丸める
    merged_rank[average_target_cols] = merged_rank[average_target_cols].round(1)

    # ========== 4. トータル行（クリック） ==========
    # クリック側の合計を出したい列を定義（最後の7列など）
    sum_cols = merged_click.columns[-7:].tolist()  # 例
    sum_values = merged_click[sum_cols].sum()
    sum_row = pd.DataFrame([sum_values], columns=sum_values.index)
    sum_row['Query'] = total_suffix
    sum_row['Query(En)'] = ' - '
    sum_row['MSV'] = ' - '
    merged_click = pd.concat([merged_click, sum_row], ignore_index=True)
    merged_click[sum_cols] = merged_click[sum_cols].astype(int)

    return merged_click, merged_rank

########################################
# 2. Example Usage (Storage, Furniture, Christmas)
########################################

df_kw_target_jp    = load_sheet_data("df_kw_target_jp")
df_kw_target_en  = load_sheet_data("df_kw_target_en")


# 2.2 Merge & process each category
df_target_jp_click, df_target_jp_rank = merge_category_data(
    df_kw_target_jp, df_click, df_rank
)
df_target_en_click, df_target_en_rank = merge_category_data(
    df_kw_target_en, df_click, df_rank
)


In [17]:

from reportlab.lib.pagesizes import A4
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from datetime import datetime, timedelta

pdfmetrics.registerFont(TTFont('GenshinGothic',
    '/content/drive/My Drive/google-colab/digital-marketing-report-generator/font/GenShinGothic-Regular.ttf'))
pdfmetrics.registerFont(TTFont('GenshinGothicBold',
    '/content/drive/My Drive/google-colab/digital-marketing-report-generator/font/GenShinGothic-Bold.ttf'))

datasets = [
    (df_click_selected, "1-a. Clicks from Top Performing Keywords", False),
    (df_imp_selected,   "1-b. Impressions from Top Performing Keywords", False),
    (df_ctr_selected,   "1-c. CTR from Top Performing Keywords", False),
    (df_rank_selected,  "1-d. Rankings from Top Performing Keywords", False),
    (df_target_jp_click, "2-a.Clicks from Target Keywords(JP)", True),
    (df_target_jp_rank,  "2-b. Rankings from Target Keywords(JP)", True),
    (df_target_en_click, "3-a.Clicks from Target Keywords(EN)", True),
    (df_target_en_rank,  "3-b. Rankings from Target Keywords(EN)", True),
]

def create_table_from_df(df, bold_last_row=False):
    # Paragraph style that wraps text
    styles = getSampleStyleSheet()
    wrap_style = ParagraphStyle(
        'WrapStyle',
        parent=styles['BodyText'],
        fontName='GenshinGothic',
        fontSize=8,
        leading=10,
        wordWrap='CJK'   # 'CJK' if mixed or Japanese text; 'LTR' if only English
    )

    # Build data rows, converting every cell to Paragraph or formatted string
    data = [df.columns.tolist()] + df.values.tolist()
    formatted_data = []

    # Identify Query columns if they exist
    query_index = df.columns.get_loc('Query') if 'Query' in df.columns else None
    query_en_index = df.columns.get_loc('Query(En)') if 'Query(En)' in df.columns else None

    for row in data:
        formatted_row = []
        for val in row:
            # Numeric formatting
            if isinstance(val, float):
                val = f"{val:,.1f}"
            elif isinstance(val, int):
                val = f"{val:,}"
            else:
                val = str(val)

            # Convert to Paragraph for wrapping in all columns
            formatted_row.append(Paragraph(val, wrap_style))
        formatted_data.append(formatted_row)

    # Set fixed widths so columns won't extend too far
    # “Query” and “Query(En)” ~120, others ~40 (adjust as needed)
    col_widths = []
    for i, col_name in enumerate(df.columns):
        if query_index is not None and i == query_index:
            col_widths.append(120)
        elif query_en_index is not None and i == query_en_index:
            col_widths.append(120)
        else:
            col_widths.append(40)

    # Create the table
    table = Table(formatted_data, repeatRows=1, colWidths=col_widths)

    # Table styling
    style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('TEXTCOLOR',  (0, 0), (-1, 0), colors.black),
        ('GRID',       (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTNAME',   (0, 0), (-1, 0), 'GenshinGothicBold'),  # Header bold
        ('FONTNAME',   (0, 1), (-1, -1), 'GenshinGothic'),     # Body regular
        ('FONTSIZE',   (0, 0), (-1, -1), 8),
        # Horizontally center every cell
        ('ALIGN',      (0, 0), (-1, -1), 'CENTER'),
        # Vertically middle every cell
        ('VALIGN',     (0, 0), (-1, -1), 'MIDDLE'),
    ])

    if bold_last_row:
        style.add('FONTNAME', (0, -1), (-1, -1), 'GenshinGothicBold')

    table.setStyle(style)
    return table

def generate_pdf():
    current_date = datetime.now().strftime('%Y-%m-%d')
    pdf = SimpleDocTemplate(
        f"gsc-keyword_performance_report_monthly-{current_date}.pdf",
        pagesize=A4
    )
    styles = getSampleStyleSheet()
    styles['Normal'].fontName = 'GenshinGothic'    # Japanese font for normal text
    styles['Title'].fontName = 'GenshinGothicBold'
    styles['Heading1'].fontName = 'GenshinGothicBold'
    styles['Heading2'].fontName = 'GenshinGothic'
    elements = []

    # --- Cover Page ---
    previous_month = datetime.now() - timedelta(days=30)
    cover_title = Paragraph("SEO Keyword Performance Report", styles['Title'])
    cover_date = Paragraph(previous_month.strftime("%B %Y"), styles['Title'])
    elements.append(Spacer(1, A4[1] / 2 - 130))
    elements.append(cover_title)
    elements.append(Spacer(1, 12))
    elements.append(cover_date)
    elements.append(Spacer(1, 200))

    # Add metadata
    current_date = datetime.now().strftime('%Y-%m-%d')
    first_day_this_month = datetime(datetime.now().year, datetime.now().month, 1)
    last_day_prev_month = first_day_this_month - timedelta(days=1)
    last_day_prev_month_str = last_day_prev_month.strftime('%Y-%m-%d')

    elements.append(Paragraph(f'Created by: Shohei on {current_date}', styles['BodyText']))
    elements.append(Paragraph('Website: heysho.com', styles['BodyText']))
    elements.append(Paragraph('Data Source: Google Search Console', styles['BodyText']))
    elements.append(Paragraph(f'Data Range: 2023-09-01 - {last_day_prev_month_str}', styles['BodyText']))
    elements.append(PageBreak())

    # Table of Contents
    elements.append(Paragraph('Table of Contents', styles['Heading1']))
    elements.append(Spacer(1, 12))
    for i, (_, title, _) in enumerate(datasets, 1):
        anchor_name = f'section_{i}'
        toc_entry = Paragraph(f'<link href="#{anchor_name}">{title}</link>', styles['Normal'])
        elements.append(toc_entry)
        elements.append(Spacer(1, 10))
    elements.append(Paragraph("**Click to jump to the page", styles['Normal']))

    elements.append(PageBreak())

    # Add each dataset with anchors
    for i, (df, title, bold_last_row) in enumerate(datasets, 1):
        anchor_name = f'section_{i}'
        elements.append(Paragraph(f'<a name="{anchor_name}"/>{title}', styles['Heading1']))
        elements.append(Spacer(1, 12))
        table = create_table_from_df(df, bold_last_row=bold_last_row)
        elements.append(table)
        elements.append(PageBreak())

    # Definitions
    elements.append(Paragraph('Definition of the Metrics', styles['Heading1']))
    elements.append(Paragraph(
        '1.Clicks ... The total number of times users have clicked on a link to a website from the Google Search Results.',
        styles['Normal']
    ))
    elements.append(Spacer(1, 7))
    elements.append(Paragraph(
        '2.Impressions ... The number of times any URL from the site appeared in search results viewed by a user.',
        styles['Normal']
    ))
    elements.append(Spacer(1, 7))
    elements.append(Paragraph(
        '3.CTR ... Click through rate is the percentage of impressions that resulted in a click.',
        styles['Normal']
    ))
    elements.append(Spacer(1, 7))
    elements.append(Paragraph(
        '4.Rankings ... The position of a website’s URL in the search engine results page.',
        styles['Normal']
    ))
    elements.append(Spacer(1, 7))
    elements.append(Paragraph(
        '5.MSV ... Monthly Search Volume is the number of times a specific search query is entered on a search engine per month.',
        styles['Normal']
    ))

    elements.append(Spacer(1, 50))
    elements.append(Paragraph('Note', styles['Heading1']))
    elements.append(Paragraph(
        'The use of this template is restricted to personal purposes only. Any commercial use or provision to third parties is strictly prohibited.Redistribution of the template, as well as the redistribution of any modified version or derivative works that incorporate modifications, is prohibited in all forms.The sale, transfer, or public use (including online sharing) of any part or the entirety of the template is also prohibited.',
        styles['BodyText']
    ))
    elements.append(Spacer(1, 14))

    # Build PDF
    pdf.build(elements)
    print("PDF report generated successfully.")

# Generate the PDF report
generate_pdf()


PDF report generated successfully.
